HW2

In [1]:
import numpy as np
import cvxpy as cp

In [2]:
class Matrix():
    __array_priority__ = 1
    
    def __init__(self, arr):
        self.n = len(arr)
        self.m = len(arr[0]) if self.n > 0 else 0
        self.ix = []
        self.iy = []
        self.v = []
        for row in range(self.n):
            assert len(arr[row]) == self.m
            for col in range(self.m):
                if arr[row][col] != 0:
                    self.ix.append(row)
                    self.iy.append(col)
                    self.v.append(arr[row][col])
    def __matmul__(self, vect):
        # M * v
        assert vect.shape[0] == self.m 
        res = [0] * self.n 
        for i in range(len(self.v)):
            res[self.ix[i]] += self.v[i] * vect[self.iy[i]]
        return np.array(res)
    
    def __rmatmul__(self, vect):
        # v * M
        assert vect.shape[0] == self.n 
        res = [0] * self.m
        for i in range(len(self.v)):
            res[self.iy[i]] += self.v[i] * vect[self.ix[i]]
        return np.array(res)

In [3]:
m = np.array([[0,2,0], [0,0,1]])
v = np.array([1,1,1])
vv = np.array([1,1])

In [4]:
Matrix(m) @ v

array([2, 1])

In [5]:
vv @ Matrix(m)

array([0, 2, 1])

In [6]:
def minimize(M, b):
    A = Matrix(M)
    At = Matrix(M.T)
    x = np.zeros(A.m)
    b = At @ b
    v = (At @ (A @ x) - b)
    d = v
    norm = np.dot(v, v)
    
    for i in range(len(b)):
        Ad = At @ (A @ d)
        alpha = norm / (d @ Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        nNorm = np.dot(v, v)

        d = v + (nNorm / norm) * d
        norm = nNorm
    return x

In [7]:
M = np.random.rand(7, 7)
b = np.random.rand(7)

res = minimize(M, b)
print("my solution:", res)

x = cp.Variable(M.shape[1])
cp.Problem(cp.Minimize(cp.sum_squares(M @ x - b)), []).solve()
print("real solution:", x.value)

print("diff:", res - x.value)

my solution: [ 18.13831288   7.7553703   -6.32481942 -12.84912904  -0.83563506
   1.22175638  -4.11015795]
real solution: [ 18.13829091   7.75531024  -6.32496528 -12.84930296  -0.8357924
   1.22167846  -4.11030434]
diff: [2.19745721e-05 6.00625067e-05 1.45859426e-04 1.73920162e-04
 1.57345037e-04 7.79142719e-05 1.46388094e-04]
